# Mavlink Camera
> Mavlink Camera Component for sending commands to a camera on a companion computer or GCS
> The server is on the companion computer and the client is on the ground station PC.

In [ ]:
#| default_exp mavlink.test_camera

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time
from mavcom.mavlink.mavcom import MAVCom, time_since_boot_ms, time_UTC_usec, date_time_str
from mavcom.mavlink.component import Component, mavutil, mavlink, MAVLink

from mavcom.mavlink.camera_client import *
from mavcom.mavlink.camera_server import *
from mavcom.utils.display import *
from fastcore.test import *

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *


### Implementation of these commands:
>
> [MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS)
[MAV_CMD_REQUEST_CAMERA_INFORMATION = 523](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION)
[MAV_CMD_REQUEST_CAMERA_SETTINGS = 524](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS)
[MAV_CMD_REQUEST_STORAGE_INFORMATION = 525](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION)
[MAV_CMD_STORAGE_FORMAT = 526](https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT)
[MAV_CMD_SET_CAMERA_ZOOM = 531](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM)
[MAV_CMD_SET_CAMERA_FOCUS = 532](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS)
[MAV_CMD_IMAGE_START_CAPTURE = 2000](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE)
[MAV_CMD_IMAGE_STOP_CAPTURE = 2001](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE)
> 
> [MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION)
[MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS)
[MAV_CMD_VIDEO_START_CAPTURE = 2500](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE)
[MAV_CMD_VIDEO_STOP_CAPTURE = 2501](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE)
[MAV_CMD_SET_CAMERA_MODE = 530](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE)
> 
**Note**
The simulated camera is implemented in PX4 [gazebo_camera_manager_plugin.cpp](https://github.com/PX4/PX4-SITL_gazebo-classic/blob/main/src/gazebo_camera_manager_plugin.cpp).


In [ ]:
#| exports
# from pymavlink.dialects.v20 import ardupilotmega as mav
# from pymavlink.dialects.v20.ardupilotmega import MAVLink


NAN = float("nan")

"""
MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
MAV_CMD_REQUEST_CAMERA_INFORMATION = 521 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
MAV_CMD_REQUEST_CAMERA_SETTINGS = 522 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
MAV_CMD_REQUEST_STORAGE_INFORMATION = 525 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
MAV_CMD_STORAGE_FORMAT = 526 # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
MAV_CMD_SET_CAMERA_ZOOM = 531 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
MAV_CMD_SET_CAMERA_FOCUS = 532 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS
MAV_CMD_IMAGE_START_CAPTURE = 2000  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
MAV_CMD_IMAGE_STOP_CAPTURE = 2001  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION
MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS
MAV_CMD_VIDEO_START_CAPTURE = 2500 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
MAV_CMD_VIDEO_STOP_CAPTURE = 2501 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
MAV_CMD_SET_CAMERA_MODE = 530 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE

"""
CAMERA_INFORMATION = mavlink.MAVLINK_MSG_ID_CAMERA_INFORMATION # https://mavlink.io/en/messages/common.html#CAMERA_INFORMATION
CAMERA_SETTINGS = mavlink.MAVLINK_MSG_ID_CAMERA_SETTINGS # https://mavlink.io/en/messages/common.html#CAMERA_SETTINGS
STORAGE_INFORMATION = mavlink.MAVLINK_MSG_ID_STORAGE_INFORMATION # https://mavlink.io/en/messages/common.html#STORAGE_INFORMATION
CAMERA_CAPTURE_STATUS = mavlink.MAVLINK_MSG_ID_CAMERA_CAPTURE_STATUS # https://mavlink.io/en/messages/common.html#CAMERA_CAPTURE_STATUS
CAMERA_IMAGE_CAPTURED = mavlink.MAVLINK_MSG_ID_CAMERA_IMAGE_CAPTURED # https://mavlink.io/en/messages/common.html#CAMERA_IMAGE_CAPTURED


### Camera Server 
> The server is on the companion computer and is used to receive commands from the camera on the ground station PC.

In [ ]:
show_doc(CameraServer)

---

### CameraServer

>      CameraServer (source_component=100, mav_type=30, camera=None,
>                    loglevel=20)

Create a mavlink Camera server Component, camera argument will normally be a  gstreamer pipeline

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| source_component | int | 100 | used for component indication |
| mav_type | int | 30 | used for heartbeat MAV_TYPE indication |
| camera | NoneType | None | camera  (or FakeCamera for testing) |
| loglevel | int | 20 | logging level |

In [ ]:
# run a server that can receive commands from a client
from mavcom.logging import LogLevels

# start a mavlink server that can receive commands from a client
with MAVCom("udpout:localhost:14445", source_system=222, loglevel=LogLevels.DEBUG) as UAV_server:
    # add the camera server components to the server
    UAV_server.add_component(CameraServer(mav_type=mavlink.MAV_TYPE_CAMERA, source_component=mavlink.MAV_COMP_ID_CAMERA, camera=None))
    UAV_server.add_component(CameraServer(mavlink.MAV_COMP_ID_CAMERA2))
    UAV_server.add_component(CameraServer(mavlink.MAV_COMP_ID_CAMERA3))
    
    time.sleep(1)


INFO   | mavcom.MAVCom      | 40.097 |  mavcom.py:379 | Thread-63 (listen) | MAVLink Mav2: True, source_system: 222
WARNIN | mavcom.CameraServe | 40.099 | camera_server.py:110 | MainThread         | Component has no camera object
INFO   | mavcom.CameraServe | 40.100 | component.py:135 | MainThread         | Component Started self.source_component = 100, self.mav_type = 30, self.source_system = 222
WARNIN | mavcom.CameraServe | 40.102 | camera_server.py:110 | MainThread         | Component has no camera object
INFO   | mavcom.CameraServe | 40.103 | component.py:135 | MainThread         | Component Started self.source_component = 101, self.mav_type = 30, self.source_system = 222
WARNIN | mavcom.CameraServe | 40.104 | camera_server.py:110 | MainThread         | Component has no camera object
INFO   | mavcom.CameraServe | 40.105 | component.py:135 | MainThread         | Component Started self.source_component = 102, self.mav_type = 30, self.source_system = 222


UAV                             
UAV                             
UAV                             


INFO   | mavcom.CameraServe | 42.101 | component.py:401 | MainThread         | CameraServer closed
INFO   | mavcom.CameraServe | 42.104 | component.py:401 | MainThread         | CameraServer closed
INFO   | mavcom.CameraServe | 42.106 | component.py:401 | MainThread         | CameraServer closed
INFO   | mavcom.MAVCom      | 42.106 |  mavcom.py:428 | MainThread         | MAVCom  closed


In [ ]:
CameraServer().list_commands()

Supported Commands: https://mavlink.io/en/messages/common.html#mav_commands
 Cmd = MAV_CMD_REQUEST_MESSAGE: 512 
 Cmd = MAV_CMD_STORAGE_FORMAT: 526 
 Cmd = MAV_CMD_SET_CAMERA_ZOOM: 531 
 Cmd = MAV_CMD_IMAGE_START_CAPTURE: 2000 
 Cmd = MAV_CMD_IMAGE_STOP_CAPTURE: 2001 
 Cmd = MAV_CMD_VIDEO_START_CAPTURE: 2500 
 Cmd = MAV_CMD_VIDEO_STOP_CAPTURE: 2501 
 Cmd = MAV_CMD_SET_CAMERA_MODE: 530 
 Cmd = MAV_CMD_VIDEO_START_STREAMING: 2502 
 Cmd = MAV_CMD_VIDEO_STOP_STREAMING: 2503 
Supported Message Requests:  https://mavlink.io/en/messages/common.html#messages

MAVLINK_MSG_ID_CAMERA_INFORMATION:  259
MAVLINK_MSG_ID_CAMERA_SETTINGS:  260
MAVLINK_MSG_ID_STORAGE_INFORMATION:  261
MAVLINK_MSG_ID_CAMERA_CAPTURE_STATUS:  262
MAVLINK_MSG_ID_CAMERA_IMAGE_CAPTURED:  263
MAVLINK_MSG_ID_VIDEO_STREAM_INFORMATION:  269
MAVLINK_MSG_ID_VIDEO_STREAM_STATUS:  270


In [ ]:
doc_class(CameraServer)

---

### CameraServer.close

>      CameraServer.close ()

Close the connection to the camera

---

### Component.count_message

>      Component.count_message (msg)

Count a message by adding it to the message_cnts dictionary. indexed by system and message type

---

### CameraServer.list_commands

>      CameraServer.list_commands ()

List the commands supported by the camera server
https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
etc

---

### CameraServer.on_mav_connection

>      CameraServer.on_mav_connection ()

Start the mavlink connection

---

### CameraServer.on_message

>      CameraServer.on_message
>                               (msg:pymavlink.dialects.v20.ardupilotmega.MAVLin
>                               k_command_long_message)

Callback for a command received from the client
This will respond to the mavlink camera and storage focused commands:

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| msg | MAVLink_command_long_message | : mavlink  Message |
| **Returns** | **bool** | **return True to indicate that the message has been handled** |

---

### Component.send_ack

>      Component.send_ack (msg, ack_result:object=0)

Send an ACK message to indicate a command was received.

---

### Component.send_command

>      Component.send_command (target_system:int, target_component:int,
>                              command_id:int, params:list, timeout=0.5)

**Note: async function** 



---

### Component.send_ping

>      Component.send_ping (target_system:int, target_component:int,
>                           ping_num:int=None)

Send self.max_pings * ping messages to test if the server is alive.

---

### Component.set_log

>      Component.set_log (loglevel)

---

### Component.set_mav_connection

>      Component.set_mav_connection (mav_com:MAVCom)

Set the mav_connection for the component

---

### Component.set_source_compenent

>      Component.set_source_compenent ()

Set the source component for the master.mav

---

### Component.set_target

>      Component.set_target (target_system, target_component)

Set the target system and component for the gimbal

---

### Component.wait_ack

>      Component.wait_ack (target_system, target_component, command_id=None,
>                          timeout=0.1)

Wait for an ack from target_system and target_component.

**Note: async function** 



---

### Component.wait_heartbeat

>      Component.wait_heartbeat (remote_mav_type=None, target_system=None,
>                                target_component=None, timeout:int=1)

Wait for a heartbeat from target_system and target_component.

**Note: async function** 





### Camera Client 
> The client is on the ground station PC and is used to send commands to the camera on the companion computer.

In [ ]:
show_doc(CameraClient)

---

### CameraClient

>      CameraClient (source_component:int, mav_type:int,
>                    loglevel:mavcom.logging.LogLevels|int=20)

Create a client component to send commands to a companion computer or GCS that will control a camera via a CameraServer instance

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| source_component | int |  | used for component indication |
| mav_type | int |  | used for heartbeat MAV_TYPE indication |
| loglevel | mavcom.logging.LogLevels \| int | 20 | logging level |

#### Example: CameraClient

In [ ]:
# run a client that can send commands to a server
with MAVCom("udpin:localhost:14445", source_system=111, loglevel=LogLevels.CRITICAL) as client: 
    # add the camera client component to the client
    gcs:CameraClient = client.add_component( CameraClient(mav_type=mavutil.mavlink.MAV_TYPE_GCS, source_component=11, loglevel=LogLevels.DEBUG) )


DEBUG  | mavcom.CameraClien | 42.274 | component.py:131 | MainThread         | set_mav_connection CameraClient component.py:131 self.mav_com = <MAVCom>
DEBUG  | mavcom.CameraClien | 42.275 | component.py:175 | Thread-73 (_thread | Starting heartbeat type: 6 to all Systems and Components
DEBUG  | mavcom.CameraClien | 42.276 | component.py:139 | MainThread         | Called from Component.start_mav_connection(), override to add startup behaviour
INFO   | mavcom.CameraClien | 42.276 | component.py:135 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | mavcom.CameraClien | 43.277 | component.py:401 | MainThread         | CameraClient closed


In [ ]:
doc_class(CameraClient)

---

### Component.close

>      Component.close ()

---

### Component.count_message

>      Component.count_message (msg)

Count a message by adding it to the message_cnts dictionary. indexed by system and message type

---

### CameraClient.image_start_capture

>      CameraClient.image_start_capture (target_system=None,
>                                        target_component=None, interval=0,
>                                        count=1)

Start image capture sequence.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| interval | int | 0 | Image capture interval |
| count | int | 1 | Number of images to capture (0 for unlimited) |

---

### CameraClient.image_stop_capture

>      CameraClient.image_stop_capture (target_system=None,
>                                       target_component=None)

Stop image capture sequence

---

### CameraClient.message_callback_cond

>      CameraClient.message_callback_cond (msg_id, target_system,
>                                          target_component, timeout=1)

Register a callback for a message received from the server
Returns the message

**Note: async function** 



---

### CameraClient.on_mav_connection

>      CameraClient.on_mav_connection ()

---

### CameraClient.on_message

>      CameraClient.on_message
>                               (msg:pymavlink.dialects.v20.ardupilotmega.MAVLin
>                               k_message)

Callback for a command received from the server

---

### CameraClient.request_message

>      CameraClient.request_message (msg_id, params=None, target_system=None,
>                                    target_component=None)

Request a message from the camera

**Note: async function** 



---

### Component.send_ack

>      Component.send_ack (msg, ack_result:object=0)

Send an ACK message to indicate a command was received.

---

### Component.send_command

>      Component.send_command (target_system:int, target_component:int,
>                              command_id:int, params:list, timeout=0.5)

**Note: async function** 



---

### CameraClient.send_message

>      CameraClient.send_message (msg)

Send a message to the camera

---

### Component.send_ping

>      Component.send_ping (target_system:int, target_component:int,
>                           ping_num:int=None)

Send self.max_pings * ping messages to test if the server is alive.

---

### CameraClient.set_camera_mode

>      CameraClient.set_camera_mode (target_system=None, target_component=None,
>                                    mode_id=0)

Set the camera mode

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| mode_id | int | 0 | https://mavlink.io/en/messages/common.html#CAMERA_MODE |

---

### CameraClient.set_camera_zoom

>      CameraClient.set_camera_zoom (target_system=None, target_component=None,
>                                    zoom_type=0, zoom_value=1)

Set the camera zoom

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| zoom_type | int | 0 |  |
| zoom_value | int | 1 | 0 to 100 zoom value |

---

### Component.set_log

>      Component.set_log (loglevel)

---

### Component.set_mav_connection

>      Component.set_mav_connection (mav_com:MAVCom)

Set the mav_connection for the component

---

### CameraClient.set_message_callback_cond

>      CameraClient.set_message_callback_cond (msg_id, target_system,
>                                              target_component)

Register a callback condition for a message received from the server

---

### Component.set_source_compenent

>      Component.set_source_compenent ()

Set the source component for the master.mav

---

### Component.set_target

>      Component.set_target (target_system, target_component)

Set the target system and component for the gimbal

---

### CameraClient.storage_format

>      CameraClient.storage_format (target_system=None, target_component=None)

Format storage (for cases where camera has storage)

---

### CameraClient.video_start_capture

>      CameraClient.video_start_capture (target_system=None,
>                                        target_component=None,
>                                        video_stream_id=0, frequency=1)

Start video capture

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video stream id (0 for all streams) |
| frequency | int | 1 | Frequency CAMERA_CAPTURE_STATUS messages should be sent while recording (0 for no messages, otherwise frequency in Hz) |

---

### CameraClient.video_start_streaming

>      CameraClient.video_start_streaming (target_system=None,
>                                          target_component=None,
>                                          video_stream_id=0)

Start video streaming

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video Stream ID (0 for all streams) |

---

### CameraClient.video_stop_capture

>      CameraClient.video_stop_capture (target_system=None,
>                                       target_component=None,
>                                       video_stream_id=0)

Stop video capture

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video stream id (0 for all streams) |

---

### CameraClient.video_stop_streaming

>      CameraClient.video_stop_streaming (target_system=None,
>                                         target_component=None,
>                                         video_stream_id=0)

Stop the video stream

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | NoneType | None |  |
| target_component | NoneType | None |  |
| video_stream_id | int | 0 | Video Stream ID (0 for all streams) |

---

### Component.wait_ack

>      Component.wait_ack (target_system, target_component, command_id=None,
>                          timeout=0.1)

Wait for an ack from target_system and target_component.

**Note: async function** 



---

### Component.wait_heartbeat

>      Component.wait_heartbeat (remote_mav_type=None, target_system=None,
>                                target_component=None, timeout:int=1)

Wait for a heartbeat from target_system and target_component.

**Note: async function** 



---

### CameraClient.wait_message_callback

>      CameraClient.wait_message_callback (cond, timeout=1)

Wait for the callback for a message received from the server

**Note: async function** 





### Example: Test locally using UDP ports

> on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`
> CameraClient is set to udpin:localhost:14445 and CameraServer is set to udpout:localhost:14445 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
> CameraClient uses async/await to send commands to the CameraServer


In [ ]:
%autoawait asyncio
import asyncio
async def main():
    MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
    MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
    
    con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
    # con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"
    with MAVCom(con1, source_system=111) as client:
        with MAVCom(con2, source_system=222) as server:
            gcs:CameraClient = client.add_component(
                CameraClient(mav_type=MAV_TYPE_GCS, source_component=11))
            # server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=cam_fake1, debug=False))
            server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=None))
    
            ret = await gcs.wait_heartbeat(remote_mav_type=mavlink.MAV_TYPE_CAMERA)
            print(f"Heartbeat received {ret = }")
    
            msg = await gcs.request_message(mavlink.MAVLINK_MSG_ID_CAMERA_INFORMATION, target_system=222, target_component=22)

            print( f"MAVLINK_MSG_ID_CAMERA_INFORMATION {msg}")
            
            # msg = await cam.request_storage_information()
            # print (msg)
            
            time.sleep(1)
            
await main()

INFO   | mavcom.MAVCom      | 43.433 |  mavcom.py:379 | Thread-75 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | mavcom.MAVCom      | 43.534 |  mavcom.py:379 | Thread-76 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | mavcom.CameraClien | 43.536 | component.py:135 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
WARNIN | mavcom.CameraServe | 43.537 | camera_server.py:110 | MainThread         | Component has no camera object
INFO   | mavcom.CameraServe | 43.538 | component.py:135 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222


UAV                             
Heartbeat received ret = (222, 22)
MAVLINK_MSG_ID_CAMERA_INFORMATION CAMERA_INFORMATION {time_boot_ms : 2248962, vendor_name : UAV, model_name : FakeCamera, firmware_version : 1, focal_length : 2.799999952316284, sensor_size_h : 3.200000047683716, sensor_size_v : 2.4000000953674316, resolution_h : 640, resolution_v : 480, lens_id : 0, flags : 0, cam_definition_version : 1, cam_definition_uri : , gimbal_device_id : 0}


INFO   | mavcom.CameraServe | 45.539 | component.py:401 | MainThread         | CameraServer closed
INFO   | mavcom.MAVCom      | 45.540 |  mavcom.py:428 | MainThread         | MAVCom  closed
INFO   | mavcom.CameraClien | 46.539 | component.py:401 | MainThread         | CameraClient closed
INFO   | mavcom.MAVCom      | 46.540 |  mavcom.py:428 | MainThread         | MAVCom  closed


In [ ]:
# | hide
# assert False, "Stop here"

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
#| exports
from mavcom.mavlink.mavcom import MAVCom
from mavcom.mavlink.component import Component, mavutil
import time

def on_message(message):
    print(f"on_message: {message}")
    return True # Return True to indicate that command was ok and send ack

class Cam1(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__(source_component=source_component, mav_type=mav_type)
        self._set_message_callback(on_message)


class Cam2(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__(source_component=source_component, mav_type=mav_type)
        self._set_message_callback(on_message)


class Cli(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__(source_component=source_component, mav_type=mav_type)
        self._set_message_callback(on_message)

In [ ]:
#| exports

async def test_client_server(con1="udpin:localhost:14445", con2="udpout:localhost:14445"):
    with MAVCom(con1, source_system=111) as client:
        with MAVCom(con2, source_system=222) as server:

            client.add_component(Cli(mav_type=mavlink.MAV_TYPE_GCS, source_component=11))
            server.add_component(Cam1(mav_type=mavlink.MAV_TYPE_CAMERA, source_component=22))
            server.add_component(Cam1(mav_type=mavlink.MAV_TYPE_CAMERA, source_component=23))

            for key, comp in client.component.items():
                # result = await comp.wait_heartbeat(target_system=222, target_component=22)
                result = await comp.wait_heartbeat(remote_mav_type=mavlink.MAV_TYPE_CAMERA, target_system=222, target_component=22)
                print(f"Component {comp}, Heartbeat: {result = }")

            Num_Iters = 3
            for i in range(Num_Iters):
                await client.component[11]._test_command(222, 22, 1)

                await client.component[11]._test_command(222, 23, 1)

            await client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")
    
    assert client.component[11].num_cmds_sent == Num_Iters * 2 + 1
    print(f"{server.component[22].message_cnts[111]['COMMAND_LONG'] = }")
    assert server.component[22].message_cnts[111]['COMMAND_LONG'] == Num_Iters
    assert client.component[11].num_acks_rcvd == Num_Iters * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == Num_Iters
    assert server.component[23].num_cmds_rcvd == Num_Iters

try:     
    import asyncio 
    # notebook does not run asyncio tasks
    asyncio.run(test_client_server(con1="udpin:localhost:14445", con2="udpout:localhost:14445"))
except: 
    pass

WARNIN | py.warnings     | 46.577 | warnings.py:109 | MainThread         | /tmp/ipykernel_9420/569135065.py:41: RuntimeWarning: coroutine 'test_client_server' was never awaited
  pass



In [ ]:
%autoawait asyncio
await test_client_server()

INFO   | mavcom.MAVCom      | 46.687 |  mavcom.py:379 | Thread-81 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | mavcom.MAVCom      | 46.788 |  mavcom.py:379 | Thread-82 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | mavcom.Cli         | 46.790 | component.py:135 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | mavcom.Cam1        | 46.792 | component.py:135 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222
INFO   | mavcom.Cam1        | 46.794 | component.py:135 | MainThread         | Component Started self.source_component = 23, self.mav_type = 30, self.source_system = 222


Component Cli, Heartbeat: result = (222, 22)
on_message: COMMAND_LONG {target_system : 222, target_component : 22, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}
on_message: COMMAND_LONG {target_system : 222, target_component : 23, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}
on_message: COMMAND_LONG {target_system : 222, target_component : 22, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}
on_message: COMMAND_LONG {target_system : 222, target_component : 23, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4 : 0.0, param5 : 0.0, param6 : 0.0, param7 : 0.0}
on_message: COMMAND_LONG {target_system : 222, target_component : 22, command : 203, confirmation : 0, param1 : 1.0, param2 : 1.0, param3 : 0.0, param4

ERROR  | mavcom.MAVCom      | 47.399 |  mavcom.py:402 | Thread-82 (listen) |  Component 24 does not exist? ; Exception: 24
WARNIN | mavcom.Cli         | 47.900 | component.py:374 | MainThread         | **No ACK: 222/24 MAV_CMD_DO_DIGICAM_CONTROL:203
INFO   | mavcom.Cam1        | 48.794 | component.py:401 | MainThread         | Cam1 closed
INFO   | mavcom.Cam1        | 49.797 | component.py:401 | MainThread         | Cam1 closed
INFO   | mavcom.MAVCom      | 49.797 |  mavcom.py:428 | MainThread         | MAVCom  closed
INFO   | mavcom.Cli         | 51.796 | component.py:401 | MainThread         | Cli closed
INFO   | mavcom.MAVCom      | 51.797 |  mavcom.py:428 | MainThread         | MAVCom  closed


server.source_system = 222;  server.message_cnts = {111: {'COMMAND_LONG': 7, 'HEARTBEAT': 1}}
client.source_system = 111;  client.message_cnts = {222: {'HEARTBEAT': 5, 'COMMAND_ACK': 6}}

client.source_system = 111 
 - comp.source_component = 11
 - comp.num_msgs_rcvd = 11
 - comp.num_cmds_sent = 7
 - comp.num_cmds_rcvd = 0
 - comp.num_acks_rcvd = 6
 - comp.num_acks_sent = 0
 - comp.num_acks_drop = 1
 - comp.message_cnts = {222: {'HEARTBEAT': 5, 'COMMAND_ACK': 6}} 

server.source_system = 222 
 - comp.source_component = 22
 - comp.num_msgs_rcvd = 4
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 3
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 3
 - comp.num_acks_drop = 0
 - comp.message_cnts = {111: {'COMMAND_LONG': 3, 'HEARTBEAT': 1}}
 - comp.source_component = 23
 - comp.num_msgs_rcvd = 4
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 3
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 3
 - comp.num_acks_drop = 0
 - comp.message_cnts = {111: {'COMMAND_LONG': 3, 'HEARTBEAT': 1}

In [ ]:
#| Hide
# assert False, "Stop here"

#### Test with Serial ports
Test using a Pixhawk connected via telemetry 2 and USB serial ports.
CamClient is set to udpin:localhost:14445 and CamServer is set to udpout:localhost:14435 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445 mavproxy.py --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("/dev/ttyACM0", source_system=111) as client:
    with MAVCom("/dev/ttyUSB0", source_system=222) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

SerialException: [Errno 2] could not open port /dev/ttyACM0: [Errno 2] No such file or directory: '/dev/ttyACM0'

> For debugging help see http://localhost:3000/tutorials/mavlink_doc&debug.html and http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev.nbdev_export()